# Nginx upstream机制的设计与实现

Nginx访问上游服务器的流程可以分为6个阶段：

1. 启动upstream机制
2. 连接上游服务器
3. 向上游服务器发送请求
4. 接收上游服务器的响应头
5. 处理接收到的响应体
6. 结束请求

## upstream机制概述

### 设计目的

什么是upstream机制？

（1）上游和下游

<img width="60%" src="https://bj.bcebos.com/ipic/Nginx-upstream机制的场景示意图.png">

（2）上游服务器提供的协议

基于事件驱动架构的upstream机制所访问是所有支持TCP的上游服务器。包括`ngx_http_proxy_module`模块基于upstream机制实现了HTTP反向代理功能，`ngx_http_memcached_module`模块基于upstream可以访问memcached服务器。

（3）每个客户端请求实际上可以向多个上游服务器发起请求

（4）反向代理与转发上游服务器响应

转发响应时要解决两个问题：

* 下游是HTTP协议，上游是基于TCP的任何协议。upstream机制会将上游响应划分为包头、包体两部分，包头部分由HTTP模块实现的process_header方法解析，包体由upstream不做任何修改地转发
* 上下游的网速可能差别比较大
    * 当上下有网速差别不大，或者下游速度更快时，nginx会开辟一块固定内存，既用来接收上游的响应，也用来把响应转发给下游
    * 当上游网速快于下游时，必须要开辟足够多的内存缓存上游响应，当内存使用打到上限时会把上游响应保存磁盘中